# AND関数の学習

## pytorchの最新版をインストールする

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
!pip install tqdm
import torch
print(torch.__version__)


## 必要なパッケージのインポート

In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn        # ネットワーク構築用
import torch.optim as optim  # 最適化関数
import torch.nn.functional as F  # ネットワーク用の様々な関数


## グローバル変数の定義

In [0]:
mb_size = 10

## ニューラルネットワークの定義

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2)  # 名前はlinear だが Ax + b のアフィン変換の形
        self.fc2 = nn.Linear(2, 2)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

## インスタンス作成

In [0]:
model = Net()  # モデルインスタンス作成

# Loss関数の指定
loss_func = nn.MSELoss()

# Optimizerの指定
optimizer = optim.Adam(model.parameters(), lr=0.1)

# データ全てのトータルロス
running_loss = 0.0

## 訓練ループ

In [0]:
# 訓練ループ
for i in range(100):
    # フィードするデータの作成
    inputs = torch.bernoulli(0.5 * torch.ones(mb_size, 2))  # 確率0.5で1となるベルヌーイ分布
    labels = torch.Tensor(mb_size, 2)
    for j in range(mb_size):
        if (inputs[j, 0] == 1.0) and (inputs[j, 1] == 1.0):
            labels[j, 0] = 1.0
            labels[j, 1] = 0.0
        else:
            labels[j, 0] = 0.0
            labels[j, 1] = 1.0

    optimizer.zero_grad()  # optimizerの初期化

    outputs = model(inputs)  # 推論計算
    loss = loss_func(outputs, labels)  # 損失関数の定義

    loss.backward()  # バックプロパゲーション
    optimizer.step()  # パラメータ更新

    # ロスの表示
    print('[%5d] loss: %.3f' % (i + 1, loss.item()))

## 学習結果の評価

In [0]:
# 性能の評価
inputs = torch.Tensor(1, 2)
inputs.data[0, 0] = 0.0
inputs.data[0, 1] = 0.0

outputs = model(inputs)
print('0 & 0 = %.4f' % (outputs.data[0, 0]))

inputs.data[0, 0] = 0.0
inputs.data[0, 1] = 1.0
outputs = model(inputs)
print('0 & 1 = %.4f' % (outputs.data[0, 0]))

inputs.data[0, 0] = 1.0
inputs.data[0, 1] = 0.0
outputs = model(inputs)
print('1 & 0 = %.4f' % (outputs.data[0, 0]))

inputs.data[0, 0] = 1.0
inputs.data[0, 1] = 1.0
outputs = model(inputs)
print('1 & 1 = %.4f' % (outputs.data[0, 0]))
